# Benchmarking skforecast

This notebook benchmarks the performance (velocity) of the `skforecast` in its different versions and keeps track of the results.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

/home/joaquin/Documents/GitHub/skforecast


In [2]:
# !pip install --upgrade numpy < 2.0 && \
# pip install --upgrade pandas && \
# pip install --upgrade scikit-learn && \
# !pip install skforecast==0.14.0

In [3]:
# Libraries
# ==============================================================================
import timeit
import time
import platform
import joblib
import os
import sys
import warnings
import hashlib
import inspect
import psutil
import sklearn
import numpy as np
import pandas as pd
import lightgbm
import skforecast
import plotly.express as px
import plotly.graph_objects as go
if skforecast.__version__ >= '0.14.0':
    from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
    from skforecast.direct import ForecasterDirect, ForecasterDirectMultiVariate
else:
    from skforecast.ForecasterAutoreg import ForecasterAutoreg
    from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
    from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
    from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.utils import *
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
print('Skforecast version: ', skforecast.__version__)

Skforecast version:  0.16.0


In [4]:
# Becnchmarking functions
# ==============================================================================
class BenchmarkRunner:
    def __init__(self, output_dir="benchmarks", repeat=10):
        self.output_dir = output_dir
        self.repeat = repeat
        os.makedirs(self.output_dir, exist_ok=True)

    def get_system_info(self):
        return {
            'datetime': pd.Timestamp.now(),
            'python_version': platform.python_version(),
            'skforecast_version': skforecast.__version__,
            'numpy_version': np.__version__,
            'pandas_version': pd.__version__,
            'sklearn_version': sklearn.__version__,
            'lightgbm_version': lightgbm.__version__,
            'platform': platform.platform(),
            'processor': platform.processor(),
            'cpu_count': psutil.cpu_count(logical=True),
            'memory_gb': round(psutil.virtual_memory().total / 1e9, 2),
        }

    def hash_function_code(self, func):
        src = inspect.getsource(func)
        return hashlib.md5(src.encode()).hexdigest()

    # def time_function(self, func, *args, **kwargs):
    #     try:
    #         timer = timeit.Timer(lambda: func(*args, **kwargs))
    #         times = timer.repeat(repeat=self.repeat, number=1)
    #         return {
    #             'avg_time': np.mean(times),
    #             'std_dev': np.std(times)
    #         }
    #     except Exception as e:
    #         warnings.warn(f"The function {func.__name__} raised an exception: {e}")
    #         return {
    #             'avg_time': float('nan'),
    #             'std_dev': float('nan')
    #         }

    def time_function(self, func, *args, **kwargs):
        times = []
        try:
            for _ in range(self.repeat):
                start = time.perf_counter()
                func(*args, **kwargs)
                end = time.perf_counter()
                times.append(end - start)

            return {
                'avg_time': np.mean(times),
                'std_dev': np.std(times),
            }
        except Exception as e:
            warnings.warn(f"The function {func.__name__} raised an exception: {e}")
            return {
                'avg_time': np.nan,
                'std_dev': np.nan
            }

    def benchmark(self, func, forecaster=None, allow_repeated_execution=True, *args, **kwargs):
        forecaster_name = type(forecaster).__name__ if forecaster else np.nan
        regressor_name = type(forecaster.regressor).__name__ if forecaster else np.nan
        func_name = func.__name__
        print(f"Benchmarking function: {func_name}")
        hash_code = self.hash_function_code(func)
        result_file = os.path.join(self.output_dir, "benchmark.joblib")

        system_info = self.get_system_info()
        timing = self.time_function(func, *args, **kwargs)

        entry = {
            'forecaster_name': forecaster_name,
            'regressor_name': regressor_name,
            'function_name': func_name,
            'function_hash': hash_code,
            'run_time_avg': timing['avg_time'],
            'run_time_std_dev': timing['std_dev'],
            **system_info
        }

        df_new = pd.DataFrame([entry])

        if os.path.exists(result_file):
            df_existing = joblib.load(result_file)

            if not allow_repeated_execution:
                cols_to_ignore = ['run_time_avg', 'run_time_std_dev', 'datetime']
                mask = (
                    df_existing.drop(columns = cols_to_ignore)
                    .eq(df_new.drop(columns = cols_to_ignore).loc[0, :])
                    .all(axis=1)
                )

                if mask.any():
                    warnings.warn("This benchmark already exists with the same hash and system info. Skipping save.")
                    return df_existing

            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
            joblib.dump(df_combined, result_file)
            return df_combined
        else:
            joblib.dump(df_new, result_file)
            return df_new
        

def plot_benchmark_results(df, function_name):
    """
    Plot benchmark results for a given function.
    """
    sorted_versions = sorted(df['skforecast_version'].unique(), key=lambda x: list(map(int, x.split('.'))))
    df['skforecast_version'] = pd.Categorical(df['skforecast_version'], categories=sorted_versions, ordered=True)
    df = df.sort_values("skforecast_version")
    medians = df.groupby('skforecast_version', observed=True)['run_time_avg'].median().reset_index()

    fig = px.strip(
        df,
        x="skforecast_version",
        y="run_time_avg",
        color="skforecast_version",
        hover_data=df.columns,
        stripmode='overlay',
    )
    fig.update_traces(jitter=True, marker=dict(size=10, opacity=0.7))

    fig.add_trace(
        go.Scatter(
            x=medians['skforecast_version'],
            y=medians['run_time_avg'],
            mode='lines+markers',
            line=dict(color='black', width=2),
            marker=dict(size=8),
            name='Median',
            showlegend=True
        )
    )

    fig.update_layout(
        title=f"Execution time of {function_name}",
        xaxis_title="skforecast version",
        yaxis_title="Execution time (seconds)",
        showlegend=True
    )
    fig.show()


In [5]:
print(f"Python version: {platform.python_version()}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"skforecast version: {skforecast.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"psutil version: {psutil.__version__}")
print("")

# Computer information
# ==============================================================================
print(f"Computer network name: {platform.node()}")
print(f"Machine type: {platform.machine()}")
print(f"Processor type: {platform.processor()}")
print(f"Platform type: {platform.platform()}")
print(f"Operating system: {platform.system()}")
print(f"Operating system release: {platform.release()}")
print(f"Operating system version: {platform.version()}")
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.12.9
scikit-learn version: 1.6.1
skforecast version: 0.16.0
pandas version: 2.2.3
numpy version: 2.2.5
psutil version: 5.9.0

Computer network name: joaquin-HP-ProBook-440-G6
Machine type: x86_64
Processor type: x86_64
Platform type: Linux-6.11.0-21-generic-x86_64-with-glibc2.39
Operating system: Linux
Operating system release: 6.11.0-21-generic
Operating system version: #21~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Feb 24 16:52:15 UTC 2
Number of physical cores: 4
Number of logical cores: 8


# ForecasterRecursiveMultiSeries

In [ ]:
# Mock data for benchmarking
# ==========================================================
n_series = 600
len_series = 2000
series_dict = {}
rng = np.random.default_rng(321)
for i in range(n_series):
    series_dict[f'series_{i}'] = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name=f'series_{i}'
    )
series_dataframe = pd.DataFrame(series_dict, index=series_dict['series_0'].index)

exog_dict = {}
rng = np.random.default_rng(321)
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=series_dict[k].index
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict[k] = exog


exog_dict_prediction = {}
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=pd.date_range(
                start=series_dict[k].index.max() + pd.Timedelta(hours=1),
                periods=100,
                freq='h'
            )
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict_prediction[k] = exog

In [7]:
# Benchmarking speed ForecasterRecursiveMultiSeries
# ==============================================================================
regressor = LGBMRegressor(random_state=8520, verbose=-1)
# regressor = LinearRegression()
if skforecast.__version__ >= '0.14.0':
    forecaster = ForecasterRecursiveMultiSeries(
        regressor=regressor,
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )
else:
     forecaster = ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )

def ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict():
    forecaster.fit(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict():
    forecaster._create_train_X_y(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog():
    forecaster.fit(series=series_dataframe)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog():
    forecaster._create_train_X_y(series=series_dataframe)

def ForecasterRecursiveMultiSeries__create_train_X_y_single_series():
    _ = forecaster._create_train_X_y_single_series(
            y = series_dict['series_0'],
            exog = exog_dict['series_0'],
            ignore_exog = False,
        )

def ForecasterRecursiveMultiSeries_predict_exog_is_dict():
    forecaster.predict(steps=100, exog=exog_dict_prediction, suppress_warnings=True)

def ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict():
    _ = forecaster._create_predict_inputs(
            steps         = 100,
            exog         = exog_dict_prediction,
            check_inputs = True
        )

def ForecasterRecursiveMultiSeries__check_predict_inputs():
    check_predict_input(
        forecaster_name  = type(forecaster).__name__,
        steps            = 100,
        is_fitted        = forecaster.is_fitted,
        exog_in_         = forecaster.exog_in_,
        index_type_      = forecaster.index_type_,
        index_freq_      = forecaster.index_freq_,
        window_size      = forecaster.window_size,
        last_window      = pd.DataFrame(forecaster.last_window_),
        exog             = exog_dict_prediction,
        exog_type_in_    = forecaster.exog_type_in_,
        exog_names_in_   = forecaster.exog_names_in_,
        interval         = None,
        levels           = forecaster.series_names_in_,
        series_names_in_ = forecaster.series_names_in_,
        encoding         = forecaster.encoding
    )


runner = BenchmarkRunner(repeat=5, output_dir="benchmarks")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog, forecaster= forecaster)
forecaster.fit(series=series_dict, exog=exog_dict)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_single_series, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_predict_exog_is_dict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__check_predict_inputs, forecaster= forecaster)

Benchmarking function: ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict
Benchmarking function: ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_single_series
Benchmarking function: ForecasterRecursiveMultiSeries_predict_exog_is_dict
Benchmarking function: ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict
Benchmarking function: ForecasterRecursiveMultiSeries__check_predict_inputs


In [11]:
# Plot results
# ==============================================================================
results_benchmark = joblib.load("benchmarks/benchmark.joblib")
results_benchmark = results_benchmark.query("forecaster_name == 'ForecasterRecursiveMultiSeries'").copy()
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'").copy()
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'").copy()
    display(df)
    plot_benchmark_results(df, function_name)

,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
16,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,13.144153,0.757586,2025-04-22 21:21:23.862947,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
36,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,12.617939,1.155304,2025-04-22 21:28:08.104547,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
48,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,12.964886,1.387282,2025-04-22 21:32:16.760560,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
60,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,19.005444,1.634159,2025-04-22 21:42:34.608069,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
72,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,20.060155,1.042921,2025-04-22 21:52:19.178605,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
84,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,19.367950,1.190324,2025-04-22 22:07:07.919896,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
120,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,19.038448,1.077365,2025-04-22 22:41:42.671386,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
132,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,17.890224,0.643582,2025-04-22 22:50:05.388907,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
168,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,17.788394,1.396641,2025-04-23 09:53:36.637667,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
180,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,19.373057,0.969650,2025-04-23 09:58:42.497417,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
17,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.087567,0.316449,2025-04-22 21:22:29.597457,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
37,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.220368,0.188786,2025-04-22 21:29:11.209381,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
49,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.952879,0.247184,2025-04-22 21:33:21.603198,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
61,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.124450,0.105336,2025-04-22 21:44:09.659069,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
73,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.537707,0.153342,2025-04-22 21:53:59.503318,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
85,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.375345,0.522490,2025-04-22 22:08:44.787750,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
121,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.412998,0.096977,2025-04-22 22:43:17.886174,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
133,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.874042,0.075614,2025-04-22 22:51:34.861877,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
169,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.157353,0.178555,2025-04-23 09:55:05.606436,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
181,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.135348,0.255934,2025-04-23 10:00:19.393957,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
18,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.731446,0.455312,2025-04-22 21:22:40.042906,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
38,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.501288,0.540289,2025-04-22 21:29:22.319034,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
50,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,14.011422,2.125178,2025-04-22 21:33:31.377188,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
62,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,16.032972,0.317394,2025-04-22 21:44:20.300867,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
74,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,17.771868,1.486137,2025-04-22 21:54:12.212377,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
86,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,17.188971,0.887911,2025-04-22 22:08:56.684592,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
122,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,17.778667,0.783816,2025-04-22 22:43:29.961188,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
134,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,15.779289,0.196799,2025-04-22 22:51:44.249800,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
170,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,18.138810,1.240769,2025-04-23 09:55:16.417345,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
182,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,17.138384,1.510580,2025-04-23 10:00:30.081392,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
19,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.600317,0.080978,2025-04-22 21:23:38.727510,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
39,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.786675,0.196242,2025-04-22 21:30:19.848705,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
51,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.888401,0.105756,2025-04-22 21:34:41.475870,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
63,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.752682,0.053015,2025-04-22 21:45:40.484984,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
75,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.609084,0.073545,2025-04-22 21:55:41.098437,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
87,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.758432,0.228057,2025-04-22 22:10:22.654278,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
123,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.979897,0.094086,2025-04-22 22:44:58.886345,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
135,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.520278,0.041948,2025-04-22 22:53:03.168299,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
171,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.902076,0.187494,2025-04-23 09:56:47.137352,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
183,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.764201,0.282018,2025-04-23 10:01:55.801746,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
20,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001374,0.000150,2025-04-22 21:23:59.663629,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
40,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001573,0.000405,2025-04-22 21:30:43.778419,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
52,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.002709,0.000407,2025-04-22 21:35:09.717123,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
64,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001653,0.000248,2025-04-22 21:46:06.719447,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
76,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001753,0.000367,2025-04-22 21:56:06.626060,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
88,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.002212,0.000881,2025-04-22 22:10:48.384325,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
124,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.002038,0.000518,2025-04-22 22:45:29.196743,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
136,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001834,0.000409,2025-04-22 22:53:28.229034,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
172,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.002325,0.000862,2025-04-23 09:57:15.377890,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
184,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.002271,0.000359,2025-04-23 10:02:25.079514,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
21,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,1.726758,0.045816,2025-04-22 21:23:59.674122,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
41,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,1.801788,0.199091,2025-04-22 21:30:43.790260,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
53,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,2.478947,0.110898,2025-04-22 21:35:09.738397,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
65,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,2.454963,0.164434,2025-04-22 21:46:06.733806,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
77,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,0.724252,0.044540,2025-04-22 21:56:06.640060,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
89,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,0.690464,0.067909,2025-04-22 22:10:48.401687,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
125,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,2.995582,0.403974,2025-04-22 22:45:29.213280,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
137,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,0.608491,0.018189,2025-04-22 22:53:28.243796,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
173,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,0.666880,0.054986,2025-04-23 09:57:15.397602,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
185,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_exog_is...,451898dccb92cc6f0a674d62eb66e4d8,0.712341,0.020307,2025-04-23 10:02:25.097923,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
22,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,1.820111,0.134371,2025-04-22 21:24:08.314669,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
42,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,1.729033,0.160180,2025-04-22 21:30:52.806205,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
54,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,2.296931,0.110852,2025-04-22 21:35:22.142101,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
66,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,2.097463,0.107805,2025-04-22 21:46:19.017155,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
78,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,0.380648,0.023189,2025-04-22 21:56:10.270907,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
90,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,0.454669,0.071564,2025-04-22 22:10:51.864724,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
126,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,2.450424,0.194692,2025-04-22 22:45:44.206531,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
138,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,0.361557,0.018010,2025-04-22 22:53:31.295536,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
174,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,0.543933,0.162940,2025-04-23 09:57:18.742364,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
186,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_predict...,52b1c12811cade5921c24c58d5503d03,0.376125,0.050791,2025-04-23 10:02:28.673536,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
23,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.310954,0.021981,2025-04-22 21:24:17.424537,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
43,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.301873,0.006419,2025-04-22 21:31:01.460036,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
55,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.492436,0.053903,2025-04-22 21:35:33.639610,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
67,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.408782,0.034458,2025-04-22 21:46:29.513457,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
79,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.113201,0.035370,2025-04-22 21:56:12.182762,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
91,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.091601,0.004678,2025-04-22 22:10:54.144804,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
127,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.483163,0.072077,2025-04-22 22:45:56.469701,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
139,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.116160,0.034849,2025-04-22 22:53:33.114403,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
175,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.355924,0.025064,2025-04-23 09:57:21.483055,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
187,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__check_predict_...,33fe67969d33981cb73558c1dc51eb82,0.134563,0.046529,2025-04-23 10:02:30.563676,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


# ForecasterRecursive

In [ ]:
# Mock data for benchmarking
# ==========================================================
len_series = 2000
rng = np.random.default_rng(321)
y = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name='y'
    )
rng = np.random.default_rng(321)
exog = pd.DataFrame(index=y.index)
exog['day_of_week'] = exog.index.dayofweek
exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
exog['month'] = exog.index.month
exog_prediction = pd.DataFrame(
                    index=pd.date_range(
                        start=series_dict[k].index.max() + pd.Timedelta(hours=1),
                        periods=100,
                        freq='h'
                    )
                 ) 
exog_prediction['day_of_week'] = exog_prediction.index.dayofweek
exog_prediction['week_of_year'] = exog_prediction.index.isocalendar().week.astype(int)
exog_prediction['month'] = exog_prediction.index.month

In [8]:
# Benchmarking speed ForecasterRecursive
# ==============================================================================
if skforecast.__version__ >= '0.14.0':
    forecaster = ForecasterRecursive(
        regressor=regressor,
        lags=50,
        transformer_y=StandardScaler(),
        transformer_exog=StandardScaler(),
    )
else:
     forecaster = ForecasterAutoreg(
        regressor=regressor,
        lags=50,
        transformer_y=StandardScaler(),
        transformer_exog=StandardScaler(),
    )

def ForecasterRecursive_fit():
    forecaster.fit(y=y, exog=exog)

def ForecasterRecursive__create_train_X_y():
    forecaster._create_train_X_y(y=y, exog=exog)


def ForecasterRecursive_predict():
    forecaster.predict(steps=100, exog=exog_prediction)

def ForecasterRecursive__create_predict_inputs():
    _ = forecaster._create_predict_inputs(
            steps        = 100,
            exog         = exog_prediction,
            check_inputs = True
        )


runner = BenchmarkRunner(repeat=10, output_dir="benchmarks")
_ = runner.benchmark(ForecasterRecursive_fit, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursive__create_train_X_y, forecaster= forecaster)
forecaster.fit(y=y, exog=exog)
_ = runner.benchmark(ForecasterRecursive_predict, forecaster= forecaster)
_ = runner.benchmark(ForecasterRecursive__create_predict_inputs, forecaster= forecaster)

Benchmarking function: ForecasterRecursive_fit
Benchmarking function: ForecasterRecursive__create_train_X_y
Benchmarking function: ForecasterRecursive_predict
Benchmarking function: ForecasterRecursive__create_predict_inputs


In [10]:
# Plot results
# ==============================================================================
results_benchmark = joblib.load("benchmarks/benchmark.joblib")
results_benchmark = results_benchmark.query("forecaster_name == 'ForecasterRecursive'").copy()
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'").copy()
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'").copy()
    display(df)
    plot_benchmark_results(df, function_name)

,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
24,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,NaN,NaN,2025-04-22 21:26:09.447315,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
26,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,NaN,NaN,2025-04-22 21:26:24.723814,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
28,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.226710,0.040298,2025-04-22 21:26:53.296630,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
32,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.190654,0.021193,2025-04-22 21:27:04.433706,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
44,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.238205,0.038765,2025-04-22 21:31:03.124401,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
56,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.375600,0.036391,2025-04-22 21:35:36.132403,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
68,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.349637,0.038892,2025-04-22 21:46:31.592093,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
80,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.383212,0.042312,2025-04-22 21:56:12.830849,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
92,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.363876,0.019510,2025-04-22 22:10:54.681237,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
128,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_fit,3a6496af8f14196a05b301212de078f0,0.392153,0.048450,2025-04-22 22:45:58.919912,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
25,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,NaN,NaN,2025-04-22 21:26:09.456096,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
27,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,NaN,NaN,2025-04-22 21:26:24.729070,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
29,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,0.002769,0.000295,2025-04-22 21:26:54.435579,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
33,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,0.002705,0.000142,2025-04-22 21:27:05.390762,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
45,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,0.003999,0.000602,2025-04-22 21:31:05.512059,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
57,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,0.005849,0.000840,2025-04-22 21:35:39.897474,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
69,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,0.005131,0.000689,2025-04-22 21:46:35.096647,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
81,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,0.004213,0.001393,2025-04-22 21:56:16.672542,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
93,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,0.005445,0.000559,2025-04-22 22:10:58.328841,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
129,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_train_X_y,287fbd1c4920496341a9d5f9b9a9aa2a,0.006034,0.001732,2025-04-22 22:46:02.856538,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
30,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,99297f20a640837258e82490d71f7b0e,NaN,NaN,2025-04-22 21:26:54.634434,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
34,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.063024,0.017257,2025-04-22 21:27:05.582838,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
46,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.069267,0.006338,2025-04-22 21:31:05.776671,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
58,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.108267,0.002877,2025-04-22 21:35:40.348862,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
70,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.136029,0.009254,2025-04-22 21:46:35.685379,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
82,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.138232,0.022104,2025-04-22 21:56:17.128241,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
94,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.121503,0.004157,2025-04-22 22:10:58.751548,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
130,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.107448,0.008185,2025-04-22 22:46:03.358586,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
142,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.100248,0.001586,2025-04-22 22:53:37.813899,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
178,ForecasterRecursive,LGBMRegressor,ForecasterRecursive_predict,4f8ae3116340ac90eb9e66eac42789de,0.168260,0.014947,2025-04-23 09:57:30.973435,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
31,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,4a58c7a370a6467cc0ba5eda400fa239,0.005099,0.000155,2025-04-22 21:26:54.641780,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
35,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,4a58c7a370a6467cc0ba5eda400fa239,0.002747,0.000366,2025-04-22 21:27:05.902025,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
47,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,4a58c7a370a6467cc0ba5eda400fa239,0.003220,0.000280,2025-04-22 21:31:06.476643,3.12.9,0.14.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
59,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,4a58c7a370a6467cc0ba5eda400fa239,0.004583,0.000859,2025-04-22 21:35:41.442892,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
71,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,4a58c7a370a6467cc0ba5eda400fa239,0.004996,0.000583,2025-04-22 21:46:37.057769,3.12.9,0.15.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
83,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,0.008883,0.003857,2025-04-22 21:56:18.525001,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
95,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,0.004467,0.001129,2025-04-22 22:10:59.978681,3.12.9,0.16.0,1.26.4,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
131,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,0.004910,0.000912,2025-04-22 22:46:04.445545,3.12.9,0.15.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
143,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,0.003969,0.000671,2025-04-22 22:53:38.827744,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
179,ForecasterRecursive,LGBMRegressor,ForecasterRecursive__create_predict_inputs,0f4a625e8b1bb7e3f0c5bb77a04cc2fb,0.005267,0.001024,2025-04-23 09:57:32.673644,3.12.9,0.16.0,1.26.4,2.2.3,1.4.2,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
